In [1]:
# importing required libraries

import sys
sys.path.append("../../")
import time
import os
import shutil
# import papermill as pm
import pandas as pd
import numpy as np
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import warnings
warnings.filterwarnings(action='ignore')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# top k items to recommend
TOP_K = 10

In [3]:
df = pd.read_csv('../data/score_board.csv')

In [4]:
df['rating'] = df['rating'].astype('float')
df = df.rename({'u_id':'userID','p_id':'itemID'},axis=1)
df = df.groupby('userID').filter(lambda x : len(x)>= 5).copy()
df

,userID,itemID,rating
13,10,892,4.0
14,10,1262,4.0
15,10,1733,3.0
16,10,2044,4.0
17,10,884,3.0
...,...,...,...
19650,5785,2128,5.0
19651,5785,16,5.0
19652,5785,1125,4.0
19653,5785,819,5.0


In [5]:
import cornac
from cornac.eval_methods import RatioSplit

In [6]:
df.values

array([[1.000e+01, 8.920e+02, 4.000e+00],
       [1.000e+01, 1.262e+03, 4.000e+00],
       [1.000e+01, 1.733e+03, 3.000e+00],
       ...,
       [5.785e+03, 1.125e+03, 4.000e+00],
       [5.785e+03, 8.190e+02, 5.000e+00],
       [5.785e+03, 8.850e+02, 5.000e+00]])

In [7]:
data = [tuple(x) for x in df.values]
data

[(10.0, 892.0, 4.0),
 (10.0, 1262.0, 4.0),
 (10.0, 1733.0, 3.0),
 (10.0, 2044.0, 4.0),
 (10.0, 884.0, 3.0),
 (10.0, 1952.0, 3.0),
 (10.0, 673.0, 4.0),
 (10.0, 1270.0, 4.0),
 (10.0, 1189.0, 4.0),
 (10.0, 902.0, 4.0),
 (10.0, 1006.0, 3.0),
 (10.0, 937.0, 4.0),
 (10.0, 158.0, 4.0),
 (10.0, 1146.0, 3.0),
 (10.0, 1402.0, 3.0),
 (10.0, 1422.0, 4.0),
 (10.0, 1373.0, 5.0),
 (10.0, 1272.0, 1.0),
 (10.0, 1934.0, 1.0),
 (10.0, 2098.0, 2.0),
 (10.0, 1880.0, 3.0),
 (10.0, 1002.0, 4.0),
 (10.0, 1125.0, 2.0),
 (10.0, 601.0, 5.0),
 (10.0, 817.0, 5.0),
 (10.0, 1882.0, 3.0),
 (10.0, 302.0, 4.0),
 (10.0, 575.0, 3.0),
 (10.0, 1904.0, 5.0),
 (10.0, 1753.0, 4.0),
 (10.0, 1317.0, 4.0),
 (10.0, 2165.0, 1.0),
 (10.0, 1050.0, 2.0),
 (10.0, 1888.0, 3.0),
 (10.0, 625.0, 3.0),
 (10.0, 1535.0, 3.0),
 (10.0, 366.0, 4.0),
 (10.0, 1257.0, 4.0),
 (10.0, 729.0, 2.0),
 (10.0, 481.0, 2.0),
 (10.0, 753.0, 3.0),
 (10.0, 2033.0, 4.0),
 (10.0, 709.0, 5.0),
 (11.0, 187.0, 5.0),
 (11.0, 2048.0, 3.0),
 (11.0, 2189.0, 5.0),
 (11.

In [8]:
# Define an evaluation method to split feedback into train and test sets
ratio_split = RatioSplit(
    data=data,
    test_size=0.20,
    rating_threshold=1.0,
    seed=123,
    exclude_unknowns=True,
    verbose=True,
)

neumf = cornac.models.NeuMF(
    num_factors=8,
    layers=[16, 8, 4],
    act_fn="tanh",
    learner="adam",
    num_epochs=200,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
    early_stopping = {'min_delta':0, 'patience':3}
)


# Instantiate evaluation metrics
ndcg = cornac.metrics.NDCG(k=TOP_K)
pre = cornac.metrics.Precision(k=TOP_K)
rec = cornac.metrics.Recall(k=TOP_K)
fm = cornac.metrics.FMeasure(k=TOP_K)

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=ratio_split,
    models=[neumf],
    metrics=[ndcg, pre, rec, fm],
).run()

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 903
Number of items = 2122
Number of ratings = 9957
Max rating = 5.0
Min rating = 1.0
Global mean = 4.0
---
Test data:
Number of users = 769
Number of items = 1193
Number of ratings = 2399
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 903
Total items = 2122

[NeuMF] Training started!




[NeuMF] Evaluation started!




TEST:
...
      |  F1@10 | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
----- + ------ + ------- + ------------ + --------- + --------- + --------
NeuMF | 0.0118 |  0.0182 |       0.0081 |    0.0293 | 2559.8239 |   1.0370



In [9]:
ncf = cornac.models.recommender.Recommender('NeuMF')

In [10]:
ncf.early_stop

<bound method Recommender.early_stop of <cornac.models.recommender.Recommender object at 0x000002E41DD8D148>>

In [12]:
tmp

,score
396,5.518555e-01
10,1.907349e-06
749,0.000000e+00
569,2.956301e-03
565,1.013279e-06
...,...
419,4.765391e-05
827,0.000000e+00
1769,0.000000e+00
829,2.464249e-07


In [13]:
neumf.rank(10)

(array([ 396,   10,  749, ..., 1769,  829, 2121], dtype=int64),
 array([5.5185550e-01, 1.9073486e-06, 0.0000000e+00, ..., 0.0000000e+00,
        2.4642486e-07, 0.0000000e+00], dtype=float32))

In [14]:
neumf.rate(10,565)

array([1.], dtype=float32)

In [15]:
len(data)

12463

In [18]:
# userID 10에 대한 추천결과
tmp = pd.DataFrame(neumf.rank(10)[1], index = neumf.rank(10)[0], columns=['prediction'])
tmp.sort_values(by=['prediction'], axis=0, ascending=False)

,prediction
1145,0.928326
924,0.856280
1688,0.835944
151,0.712507
1089,0.695450
...,...
1950,0.000000
2028,0.000000
1874,0.000000
1584,0.000000
